<div style=" background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px 0; color:#fff; text-align: center">Data & tests statistiques
</h2>
</div>

In [21]:
import pandas as pd
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt 
import seaborn as sns

from scipy.stats import ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import chi2
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr

In [22]:
df_stat = pd.read_csv('../Source/Data_processed/lapage_all.csv', sep=';', parse_dates=True)

In [23]:
df_stat

,id_prod,Timestamp,session_id,client_id,sex,birth,price,categ
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,f,1967,11.99,0
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,m,1960,19.37,0
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,m,1988,4.50,0
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,f,1989,6.55,0
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,f,1956,16.49,0
...,...,...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,f,1996,21.92,1
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,f,1994,48.99,2
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,f,1985,26.99,1
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,m,1953,8.99,0


In [24]:
df_stat.nunique()

id_prod         3265
Timestamp     687534
session_id    345505
client_id       8600
sex                2
birth             76
price           1442
categ              3
dtype: int64

In [25]:
df_stat.describe()

,birth,price,categ
count,687534.000000,687534.000000,687534.000000
mean,1977.817391,17.493918,0.448789
std,13.607935,18.238337,0.594563
min,1929.000000,0.620000,0.000000
25%,1970.000000,8.990000,0.000000
50%,1980.000000,13.990000,0.000000
75%,1987.000000,19.080000,1.000000
max,2004.000000,300.000000,2.000000


In [26]:
# Graphe en ligne du chiffre d'affaires dans le temps
df_stat.resample('ME', )['price'].sum().plot(kind="line", style="-b", figsize=(10, 6))
plt.title("Évolution du chiffre d'affaires dans le temps")
plt.xlabel("Date")
plt.ylabel("Chiffre d'affaires", )
plt.grid(True)
plt.show()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [ ]:
#month_resample = 

# Moyenne Mobile sur 30 jours
df_stat['Moyenne mobile à 30 jours'] = df_stat.resample('ME').sum(['price']).rolling(30).mean()

# Moyenne Mobile sur 7 jours
df_stat['Moyenne mobile à 7 jours'] = df_stat.resample('ME').sum(['price']).rolling(7).mean()

# Variable à représenter
vars = ['price', 'Moyenne mobile à 30 jours', 'Moyenne mobile à 7 jours' ]

df_stat[vars].plot()

ValueError: Cannot set a DataFrame with multiple columns to the single column Moyenne mobile à 30 jours